# Capston Project 

In [1]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

In [2]:
# Global Variables

background = None
accumulated_weight = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [3]:
#function to find average background value

def cal_accum_avg(frame,accumulated_weight):
    
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        return None
    
    cv2.accumulateWeighted(frame,background,accumulated_weight)

In [4]:
#function for segment in ROI

def segment(frame,threshold = 25):
    
    diff = cv2.absdiff(background.astype('uint8'),frame)
    
    ret,thresholded_img = cv2.threshold(diff,threshold,255,
                                       cv2.THRESH_BINARY)
    
    image,contours,hierarchy = cv2.findContours(thresholded_img.copy(),
                                               cv2.RETR_EXTERNAL,
                                               cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    
    else:
        #Assuming the largest external contour in roi is the hand
        
        hand_segment = max(contours,key = cv2.contourArea)
        
        return (thresholded_img,hand_segment)

In [5]:
#Function to count fingers using Convexhull

def count_fingers(thresholded_img,hand_segment):
    
    conv_hull = cv2.convexHull(hand_segment)
    
    #most extreme top,bottom,left and right points
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    #finding center point
    cx = (left[0] + right[0]) // 2
    cy = (top[1] + bottom[1]) // 2
    
    #calculate distance from center to all extreme points
    distance = pairwise.euclidean_distances([(cx,cy)],
                                           Y=[left,right,top,bottom])[0]
    
    #calculate one of the max distance
    max_distance = distance.max()
    
    #create circle 
    radius = int(0.8 * max_distance)
    circumference = (2 * np.pi * radius)
    
    circular_roi = np.zeros(thresholded_img.shape[:2],dtype='uint8')
    
    cv2.circle(circular_roi,(cx,cy),radius,255,10)
    
    circular_roi = cv2.bitwise_and(thresholded_img,thresholded_img,
                                  mask = circular_roi)
    
    image,contours,hierarchy= cv2.findContours(circular_roi.copy(),
                                               cv2.RETR_EXTERNAL,
                                        cv2.CHAIN_APPROX_NONE)
    
    count = 0
    
    for cnt in contours:
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        out_of_wrist = ((cy + (cy * 0.25)) > (y + h))
        
        limit_points = ((circumference * 0.25) > cnt.shape[0])
        
        if out_of_wrist and limit_points:
            count += 1
            
    return count

In [6]:
cam = cv2.VideoCapture(0)

num_frames = 0

while True:
    
    ret,frame = cam.read()
    
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()
    
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    
    gray_blur = cv2.GaussianBlur(gray,(7,7),0)
    
    if num_frames < 60:
        cal_accum_avg(gray_blur,accumulated_weight)
        
        if num_frames <= 59:
            cv2.putText(frame_copy,'WAIT, GETTING BACKGROUND',(200,300),
                       cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
            cv2.imshow('finger count',frame_copy)
            
    else:
        hand = segment(gray_blur)
        
        if hand is not None:
            
            thresholded, hand_segment = hand
            
            #draw contours around real hand in live stream
            cv2.drawContours(frame_copy,
                             [hand_segment+[roi_right,roi_top]],
                            -1,
                            (0,0,255),
                            5)
            fingers = count_fingers(thresholded,hand_segment)

            
            cv2.putText(frame_copy,
                       str(fingers),
                       (70,50),
                       cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                       (255,0,0),
                       2)
            
            cv2.imshow('thresholded',thresholded)
            
    cv2.rectangle(frame_copy,
                 (roi_left,roi_top),
                 (roi_right,roi_bottom),
                 (0,255,0),
                 5)
    
    num_frames += 1
    
    cv2.imshow('finger count', frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
        
cv2.destroyAllWindows()
cam.release()